In [1]:
import cdsapi
from pathlib import Path


# ERA5 Data Download

This notebook downloads daily atmospheric data from the Copernicus Climate Data Store (CDS) for Arctic sea ice forecasting.

## Configuration

Define the ERA5 variables, time range, and spatial extent for download. The Arctic region is bounded by 55°N-90°N latitude.

In [2]:
dataset = "derived-era5-single-levels-daily-statistics"
variables = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "mean_sea_level_pressure",
    "total_precipitation",
]
days = [day for day in range(1, 32)]
years = range(1979, 2024)
months = range(1, 13)

base_request = {
    "product_type": "reanalysis",
    "variable": variables,
    "day": days,
    "daily_statistic": "daily_mean",
    "time_zone": "utc+00:00",
    "frequency": "6_hourly",
    "area": [90, -180, 55, 180],
    "format": "netcdf",
}

project_root = Path.cwd()
if project_root.name == "notebooks":
    project_root = project_root.parent

raw_root = project_root / "data" / "raw" / "era5"


## Download Loop

Iterates through years, months, and variables to download ERA5 data. Implements retry logic with exponential backoff for robustness.

In [3]:
import time

client = cdsapi.Client()

raw_root.mkdir(parents=True, exist_ok=True)

for year in years:
    year_dir = raw_root / f"{year}"
    year_dir.mkdir(parents=True, exist_ok=True)
    for month in months:
        month_str = f"{month:02d}"
        for variable in variables:
            var_name = variable.replace("_", "").replace("component", "comp").replace("of", "")
            target_path = year_dir / f"era5_{year}_{month_str}_{var_name}.nc"

            if target_path.exists():
                print(f"Skipping existing file: {target_path.name}")
                continue

            request = base_request.copy()
            request["year"] = str(year)
            request["month"] = [month_str]
            request["variable"] = [variable]

            max_retries = 5
            initial_backoff = 60  # seconds

            for attempt in range(max_retries):
                try:
                    print(
                        f"Requesting ERA5 data for {variable} {year}-{month_str} (attempt {attempt + 1}/{max_retries})")
                    client.retrieve(dataset, request, target=str(target_path))
                    print(f"Successfully downloaded {target_path.name}")
                    break  # Exit retry loop on success
                except Exception as e:
                    print(f"Download failed on attempt {attempt + 1}: {e}")
                    if attempt < max_retries - 1:
                        backoff_time = initial_backoff * (2 ** attempt)
                        print(f"Retrying in {backoff_time} seconds...")
                        time.sleep(backoff_time)
                    else:
                        print(f"Failed to download {target_path.name} after {max_retries} attempts. Skipping file.")


2025-09-27 23:06:45,766 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-27 23:06:45,767 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


Skipping existing file: era5_1979_01_10mucompwind.nc
Skipping existing file: era5_1979_01_10mvcompwind.nc
Skipping existing file: era5_1979_01_2mtemperature.nc
Skipping existing file: era5_1979_01_meansealevelpressure.nc
Skipping existing file: era5_1979_01_totalprecipitation.nc
Skipping existing file: era5_1979_02_10mucompwind.nc
Skipping existing file: era5_1979_02_10mvcompwind.nc
Skipping existing file: era5_1979_02_2mtemperature.nc
Skipping existing file: era5_1979_02_meansealevelpressure.nc
Skipping existing file: era5_1979_02_totalprecipitation.nc
Skipping existing file: era5_1979_03_10mucompwind.nc
Skipping existing file: era5_1979_03_10mvcompwind.nc
Skipping existing file: era5_1979_03_2mtemperature.nc
Skipping existing file: era5_1979_03_meansealevelpressure.nc
Skipping existing file: era5_1979_03_totalprecipitation.nc
Skipping existing file: era5_1979_04_10mucompwind.nc
Skipping existing file: era5_1979_04_10mvcompwind.nc
Skipping existing file: era5_1979_04_2mtemperature.nc


2025-09-27 23:06:47,383 INFO Request ID is 5b50d885-113d-4e1b-8860-a0d828e23ef1
2025-09-27 23:06:47,498 INFO status has been updated to accepted
2025-09-27 23:06:56,128 INFO status has been updated to running
2025-09-27 23:07:21,284 INFO status has been updated to successful


1c8bb7595573851b4834f52eeea1d73d.nc:   0%|          | 0.00/8.71M [00:00<?, ?B/s]

Successfully downloaded era5_1984_09_meansealevelpressure.nc
Skipping existing file: era5_1984_09_totalprecipitation.nc
Skipping existing file: era5_1984_10_10mucompwind.nc
Skipping existing file: era5_1984_10_10mvcompwind.nc
Skipping existing file: era5_1984_10_2mtemperature.nc
Skipping existing file: era5_1984_10_meansealevelpressure.nc
Skipping existing file: era5_1984_10_totalprecipitation.nc
Skipping existing file: era5_1984_11_10mucompwind.nc
Skipping existing file: era5_1984_11_10mvcompwind.nc
Skipping existing file: era5_1984_11_2mtemperature.nc
Skipping existing file: era5_1984_11_meansealevelpressure.nc
Skipping existing file: era5_1984_11_totalprecipitation.nc
Skipping existing file: era5_1984_12_10mucompwind.nc
Skipping existing file: era5_1984_12_10mvcompwind.nc
Skipping existing file: era5_1984_12_2mtemperature.nc
Skipping existing file: era5_1984_12_meansealevelpressure.nc
Skipping existing file: era5_1984_12_totalprecipitation.nc
Skipping existing file: era5_1985_01_10m